In [1]:
import os
import json
from shapely.geometry import Polygon
from typing import List, Dict

In [2]:
def calcular_iou(poligono1: Polygon, poligono2: Polygon) -> float:
    interseccion = poligono1.intersection(poligono2).area
    union = poligono1.union(poligono2).area
    return interseccion / union

def nms(shapes: List[Dict], iou_threshold: float = 0.8) -> List[Dict]:
    # Convertir puntos a polígonos de Shapely
    poligonos = [Polygon(shape['points']) for shape in shapes]
    indices_descartados = set()

    for i in range(len(poligonos)):
        if i in indices_descartados:
            continue

        for j in range(i + 1, len(poligonos)):
            try:
                if calcular_iou(poligonos[i], poligonos[j]) > iou_threshold:
                    indices_descartados.add(j)
            except Exception as e:
                pass

    # Filtrar las formas que no han sido descartadas
    shapes_filtradas = [shape for i, shape in enumerate(shapes) if i not in indices_descartados]
    
    return shapes_filtradas

In [3]:
labels = []
images_paths = []
labels_dict = {}
remove_labels_list = []

# labels_dict = {'cow': 'sheep', 'horse': 'sheep', 'bear': 'sheep'}
# remove_labels_list = ['zebra', 'bird', 'person', 'boat']

# Función para procesar todos los archivos .json en un directorio dado
def process_json_files_in_directory(directory, starts_with):
  for root, dirs, files in os.walk(directory):
    for file in files:
      if file.endswith('.json') and file.startswith(starts_with):
        file_path = os.path.join(root, file)        
        with open(file_path, 'r', encoding='utf-8') as json_file:
          data = json.load(json_file)
        # Filtrar elementos en 'shapes' y reemplazar etiquetas según sea necesario
        # Verificar si 'shapes' está en el archivo JSON
        # Eliminar las anotaciones vacías, points = []
        if 'shapes' in data:
          # Filtrar eliminando los que están en la lista remove_labels_list y tener al menos 4 puntos para calcular área.
          data['shapes'] = [shape for shape in data['shapes'] if shape['label'] not in remove_labels_list and len(shape['points']) > 4]
          # Descartar los duplicados en una región utilizando non maximum suppression con iou
          data['shapes'] = nms(data['shapes'], iou_threshold=0.7)
          # Iterar sobre cada elemento en 'shapes'
          for shape in data['shapes']:
            # Aplicar reemplazo utilizando preprocessing_text
            # Reemplaza el valor de la etiqueta según el diccionario labels_dict
            # Si label_value no está en labels_dict, devuelve label_value sin cambios
            if 'label' in shape:
              shape['label'] = labels_dict.get(shape['label'], shape['label'])
              
            # si no es un labels conocido, imprimir la imagen por la pantalla para decidir si lo convierte o elimina a posteriori.
            if shape['label'] not in labels and shape['label'] not in labels_dict.keys():
              image_path = data['imagePath']
              images_paths.append(image_path)
              labels.append(shape['label'])

      
        # Guardar el contenido modificado de nuevo en el archivo JSON
        with open(file_path, 'w', encoding='utf-8') as json_file:
          json.dump(data, json_file, ensure_ascii=False, indent=4)

In [4]:
directory_path = './Junio/processing'
process_json_files_in_directory(directory_path, starts_with='')
for label, path in zip(labels, images_paths):
  print(f'{label} in path: {path}')

sheep in path: 2_000000.jpg
dog in path: 2_000264.jpg
